In [1]:
!pip install yt_dlp pydub git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-uifahmrw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-uifahmrw
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 4.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 58.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 5.8 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803669 sha256=d5e1131c972196954dad868070e9ebaf6cc47a76616c099a28b9fafcdb86fb9f
  Stored in directory: /tmp/pip-ephem-wheel-cache-1xk3txdy/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Su

In [2]:
import os
import yt_dlp
import pandas as pd
import whisper
import cv2
import numpy as np
from pydub import AudioSegment
from moviepy.editor import VideoFileClip
from pathlib import Path

In [3]:
def is_video_available(video_id):
    """Verifica se o vídeo está disponível no YouTube."""
    video_url = f"https://www.youtube.com/watch?v={video_id}"
    ydl_opts = {'quiet': True, 'noplaylist': True}
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            info = ydl.extract_info(video_url, download=False)
            return True if info else False
        except yt_dlp.utils.DownloadError:
            return False

def download_video(video_id, output_path):
    """Baixa o vídeo do YouTube e salva no diretório especificado."""
    video_url = f"https://www.youtube.com/watch?v={video_id}"
    ydl_opts = {
        'format': 'bestvideo+bestaudio',
        'merge_output_format': 'mp4',
        'outtmpl': output_path
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

def cut_video(input_path, output_path, start_time, end_time):
    """Recorta um trecho do vídeo usando moviepy."""
    try:
        start_time, end_time = float(start_time), float(end_time)
        video = VideoFileClip(input_path).subclip(start_time, end_time)
        video.write_videofile(output_path, codec="libx264", audio_codec="aac")
    except Exception as e:
        print(f"Erro ao cortar o vídeo: {e}")

def extract_frames(video_path, output_folder, fps=1):
    """Extrai frames do vídeo a cada 'fps' segundos usando OpenCV."""
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(frame_rate / fps)
    count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f'frame_{count:04d}.jpg')
            cv2.imwrite(frame_filename, frame)
        count += 1
    
    cap.release()
    cv2.destroyAllWindows()

def extract_audio(video_path, audio_path):
    """Extrai o áudio do vídeo e converte para WAV usando pydub."""
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)
    sound = AudioSegment.from_file(audio_path, format="mp4")
    sound.export(audio_path, format="wav")

def transcribe_audio(audio_path):
    """Transcreve o áudio usando Whisper."""
    try:
        model = whisper.load_model("base")
        result = model.transcribe(audio_path)
        return result["text"]
    except AttributeError:
        raise ImportError("Erro ao carregar o modelo Whisper. Certifique-se de que a biblioteca está instalada corretamente com 'pip install openai-whisper'.")

In [4]:
if __name__ == "__main__":
    csv_path = "/kaggle/input/speech/avspeech_train.csv"
    df = pd.read_csv(csv_path, header=None)
    
    output_dir = "avspeech_data"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    failed_videos = []
    
    for index, row in df.iterrows():
        video_id, start_time, end_time = row[0], row[1], row[2]
        
        if not is_video_available(video_id):
            print(f"Vídeo {video_id} não disponível. Pulando...")
            failed_videos.append(video_id)
            continue
        
        video_path = os.path.join(output_dir, f"video_{video_id}.mp4")
        clip_path = os.path.join(output_dir, f"clip_{video_id}.mp4")
        audio_path = os.path.join(output_dir, f"audio_{video_id}.wav")
        frames_folder = os.path.join(output_dir, f"frames_{video_id}")
        
        print(f"Baixando vídeo {video_id}...")
        download_video(video_id, video_path)
        
        print("Cortando vídeo...")
        cut_video(video_path, clip_path, start_time, end_time)
        
        print("Extraindo frames...")
        extract_frames(clip_path, frames_folder)
        
        print("Extraindo áudio...")
        extract_audio(clip_path, audio_path)
        
        print("Transcrevendo áudio...")
        transcription = transcribe_audio(audio_path)
        print("Transcrição:", transcription)
    
    if failed_videos:
        with open(os.path.join(output_dir, "failed_videos.txt"), "w") as f:
            f.write("\n".join(failed_videos))
        print("Lista de vídeos indisponíveis salva em 'failed_videos.txt'.")

ERROR: [youtube] CJoOwXcjhds: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Vídeo CJoOwXcjhds não disponível. Pulando...
Baixando vídeo AvWWVOgaMlk...
[youtube] Extracting URL: https://www.youtube.com/watch?v=AvWWVOgaMlk
[youtube] AvWWVOgaMlk: Downloading webpage
[youtube] AvWWVOgaMlk: Downloading tv client config
[youtube] AvWWVOgaMlk: Downloading player e7567ecf
[youtube] AvWWVOgaMlk: Downloading tv player API JSON
[youtube] AvWWVOgaMlk: Downloading ios player API JSON
[youtube] AvWWVOgaMlk: Downloading m3u8 information
[info] AvWWVOgaMlk: Downloading 1 format(s): 136+140
[download] Destination: avspeech_data/video_AvWWVOgaMlk.f136.mp4
[download] 100% of  240.74MiB in 00:00:35 at 6.72MiB/s     
[download] Destination: avspeech_data/video_AvWWVOgaMlk.f140.m4a
[download] 100% of   19.66MiB in 00:00:07 at 2.59MiB/s     
[Merger] Merging formats into "avspeech_data/video_AvWWVOgaMlk.mp4"
Deleting original file avspeech_data/video_AvWWVOgaMlk.f140.m4a (pass -k to keep)
Deleting original file avspeech_data/video_AvWWVOgaMlk.f136.mp4 (pass -k to keep)
Cortando víde

MoviePy - Done.
Moviepy - Writing video avspeech_data/clip_AvWWVOgaMlk.mp4



Moviepy - Done !
Moviepy - video ready avspeech_data/clip_AvWWVOgaMlk.mp4
Extraindo frames...
Extraindo áudio...
MoviePy - Writing audio in avspeech_data/audio_AvWWVOgaMlk.wav


MoviePy - Done.
Transcrevendo áudio...


...

MoviePy - Done.
Moviepy - Writing video avspeech_data/clip_gBNFfez-vUU.mp4



Moviepy - Done !
Moviepy - video ready avspeech_data/clip_gBNFfez-vUU.mp4
Extraindo frames...
Extraindo áudio...
MoviePy - Writing audio in avspeech_data/audio_gBNFfez-vUU.wav


MoviePy - Done.
Transcrevendo áudio...


Transcrição:  и вот плюшки можно будет снимать. Ну, понятно, что это всё.
Baixando vídeo ipcZj9l2a6o...
[youtube] Extracting URL: https://www.youtube.com/watch?v=ipcZj9l2a6o
[youtube] ipcZj9l2a6o: Downloading webpage
[youtube] ipcZj9l2a6o: Downloading tv client config
[youtube] ipcZj9l2a6o: Downloading player e7567ecf
[youtube] ipcZj9l2a6o: Downloading tv player API JSON
[youtube] ipcZj9l2a6o: Downloading ios player API JSON
[youtube] ipcZj9l2a6o: Downloading m3u8 information
[info] ipcZj9l2a6o: Downloading 1 format(s): 137+140
[download] Destination: avspeech_data/video_ipcZj9l2a6o.f137.mp4
[download]  70.7% of  165.91MiB at   40.49MiB/s ETA 00:01  

KeyboardInterrupt: 